<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-packages" data-toc-modified-id="Import-packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import packages</a></span></li><li><span><a href="#set-parameters" data-toc-modified-id="set-parameters-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>set parameters</a></span></li><li><span><a href="#calculation" data-toc-modified-id="calculation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>calculation</a></span></li><li><span><a href="#solve-linear-program-maximizing-expected-profit" data-toc-modified-id="solve-linear-program-maximizing-expected-profit-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>solve linear program maximizing expected profit</a></span></li><li><span><a href="#Computation-of-Cost,-Profit,-Risk" data-toc-modified-id="Computation-of-Cost,-Profit,-Risk-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Computation of Cost, Profit, Risk</a></span></li></ul></div>

## Import packages

In [1]:
import numpy as np
from scipy.optimize import linprog

A stock is at 100. 
In 6-months time, your best guess is that the stock could take any of the five values 150, 130, 100, 80 or 70 with equal probability. You are considering investing in the stock along with 6-month call options with strikes 80, 100 and 120 with prices 25, 10 and 5 respectively. You may also invest in a risk-free zero coupon bond with FV 100 and maturity six months that is trading at $99. Your budget is 100,000. Also, you do not want to sell more than 5000 options at any time.

Calculate your sharpe ratio with long only investments in the stock and bond. Sharpe ratio (for our purposes) is the ratio of return to risk (sigma).

Then re-calculate your sharpe ratio with long only investments in stock and bond and options positions with short positions limited to 5000 in each option.

1. What are the expected profits with and without options included?

2. What are the Sharpe ratios with and without options included?

You may use MATLAB, Python or other software if you prefer. You may use the code (in MATLAB and Python) that is available on canvas and adapt it suitably.

## set parameters

In [2]:
# Get Prices and future prices, each has 1/3 probability 
stock_price = 100
stock_prices_future = np.transpose(np.array([[150,130,100,80,70]]))
bond_price = 99
bond_prices_future = np.transpose(np.array([[100,100,100,100,100]]))

option1_price = 25
option2_price = 10
option3_price = 5
option1_prices_future = np.transpose(np.array([[70,50,20,0,0]])) #strike 80
option2_prices_future = np.transpose(np.array([[50,30,0,0,0]])) #strike 100
option3_prices_future = np.transpose(np.array([[30,10,0,0,0]])) #strike 120

## calculation

In [3]:
# Get expected profits
expected_profit_stock = np.mean(stock_prices_future-stock_price)
expected_profit_bond = 10
expected_profit_option1 = np.mean(option1_prices_future - option1_price)
expected_profit_option2 = np.mean(option2_prices_future - option2_price)
expected_profit_option3 = np.mean(option3_prices_future - option3_price)


# get returns for stock , bond and options
stock_returns = (stock_prices_future - stock_price)/stock_price;
bond_returns = (bond_prices_future - bond_price)/bond_price;
option1_returns =(option1_prices_future - option1_price)/option1_price;
option2_returns =(option2_prices_future - option2_price)/option2_price;
option3_returns =(option3_prices_future - option3_price)/option3_price;

# expected returns for stock
expected_stock_return = np.mean(stock_returns)
expected_bond_return = np.mean(bond_returns)
expected_option1_return = np.mean(option1_returns)
expected_option2_return = np.mean(option2_returns)
expected_option3_return = np.mean(option3_returns)

print('expected_stock_return: ',expected_stock_return)
print('expected_bond_return: ',expected_bond_return)
print('expected_option1_return: ',expected_option1_return)
print('expected_option2_return: ',expected_option2_return)
print('expected_option3_return: ',expected_option3_return)

expected_stock_return:  0.06000000000000002
expected_bond_return:  0.010101010101010102
expected_option1_return:  0.11999999999999993
expected_option2_return:  0.6
expected_option3_return:  0.6


## solve linear program maximizing expected profit

In [4]:
# define the function to maximize
# why negative here? Because linprog is used to minimize
f_bond_stock = [-1*expected_profit_stock, -1*expected_profit_bond]
A_bst = [[100, 99]]
b_bst = [100000]
x0_bounds = (0, None)
x1_bounds = (0, None)

result_bst = linprog(f_bond_stock, A_ub=A_bst, b_ub=b_bst, bounds=[x0_bounds, x1_bounds])

In [5]:
# define the function to maximize
# why negative here? Because linprog is used to minimize
f = [-1*expected_profit_stock, -1*expected_profit_bond, -expected_profit_option1 ,-expected_profit_option2, -expected_profit_option3]
A = [[100, 99, 25, 10, 5]]
b = [100000]
x0_bounds = (0, None)
x1_bounds = (0, None)
x2_bounds = (-5000, None)
x3_bounds = (-5000, None)
x4_bounds = (-5000, None)
result = linprog(f, A_ub=A, b_ub=b, bounds=[x0_bounds, x1_bounds, x2_bounds, x3_bounds, x4_bounds])

In [6]:
print(result_bst)

     con: array([], dtype=float64)
     fun: -10101.010068711184
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([0.00031525])
  status: 0
 success: True
       x: array([1.11178211e-07, 1.01010101e+03])


In [7]:
print(result)

     con: array([], dtype=float64)
     fun: -119999.93079558674
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([0.10917305])
  status: 0
 success: True
       x: array([ 6.39120304e-06,  1.00814660e-05, -4.99999976e+03,  2.24971789e+04,
        5.61888483e+00])


## Computation of Cost, Profit, Risk

In [8]:
expected_profit_bst =  -np.dot(np.array(f_bond_stock),np.array(result_bst.x));
expected_profit_bst

10101.010068711184

In [9]:
expected_profit_option =  -np.dot(np.array(f),np.array(result.x));
expected_profit_option

119999.93079558674

In [10]:
# computation of Risk
future_prices = np.transpose(np.array([stock_prices_future.reshape(5),np.squeeze(bond_prices_future), 
                                       np.squeeze(option1_prices_future),  np.squeeze(option2_prices_future), np.squeeze(option3_prices_future)]))
future_prices # Five asset's future price                             

array([[150, 100,  70,  50,  30],
       [130, 100,  50,  30,  10],
       [100, 100,  20,   0,   0],
       [ 80, 100,   0,   0,   0],
       [ 70, 100,   0,   0,   0]])

In [11]:
np.array([stock_price,bond_price])

array([100,  99])

In [12]:
cost = np.dot(np.array([stock_price,bond_price, option1_price, option2_price, option3_price]), np.array(result.x))
print(cost)
portfolio_prices_future = np.dot(future_prices, np.array(result.x))
print('portfolio_prices_future:  ',portfolio_prices_future)

portfolio_returns = (portfolio_prices_future- cost)/cost
print('portfolio_returns:  ',portfolio_returns)

99999.89082694684
portfolio_prices_future:   [ 7.75027529e+05  4.24971569e+05 -9.99999936e+04  1.51944284e-03
  1.45553081e-03]
portfolio_returns:   [ 6.75028375  3.24972033 -2.00000103 -0.99999998 -0.99999999]


In [13]:
# In python np.std use n as denominator not n-1, matlab std use n-1 that's where the difference! 
mu = np.mean(portfolio_returns)
sigma = np.std(portfolio_returns)
 
# calculate sharpe!
sharpe = mu/sigma
print('sharpe ratio with options', sharpe)

sharpe ratio with options 0.36205411992987174


In [14]:
cost_bst = np.dot(np.array([stock_price,bond_price]), np.array(result_bst.x))
print(cost_bst)
future_prices_bst = np.transpose(np.array([stock_prices_future.reshape(5),np.squeeze(bond_prices_future)]))
portfolio_prices_future_bst = np.dot(future_prices_bst, np.array(result_bst.x))
print('portfolio_prices_future:  ',portfolio_prices_future_bst)

portfolio_returns_bst = (portfolio_prices_future_bst- cost_bst)/cost_bst
print('portfolio_returns:  ',portfolio_returns_bst)

mu_bst = np.mean(portfolio_returns_bst)
sigma_bst = np.std(portfolio_returns_bst)
 
# calculate sharpe!
sharpe_bst = mu_bst/sigma_bst
print('sharpe ratio with only Bonds and Stocks', sharpe_bst)

99999.99968475457
portfolio_prices_future:   [101010.10069712 101010.10069489 101010.10069156 101010.10068934
 101010.10068822]
portfolio_returns:   [0.01010101 0.01010101 0.01010101 0.01010101 0.01010101]
sharpe ratio with only Bonds and Stocks 302176990.91817033
